In [1]:
import torch
import pypose as pp

# 1. Intialization

In [2]:
a = pp.so3(torch.randn(2,3))
x = pp.identity_SE3(2,1)
y = pp.randn_se3(2,2)
print('a:', a, '\nx.shape:', x.shape, '\nx.gshape:', x.gshape)
print(x.gview(2))
print(y)

a: so3Type Group:
tensor([[-2.8550,  0.9274,  1.9018],
        [-0.3818, -1.8837,  0.5071]]) 
x.shape: torch.Size([2, 1, 7]) 
x.gshape: torch.Size([2, 1])
SE3Type Group:
tensor([[0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 1.]])
se3Type Group:
tensor([[[ 0.7216,  0.3705,  0.7914,  0.5841, -0.8191, -0.2729],
         [-0.7464, -0.3715,  0.9468, -1.5123,  0.0142, -0.0490]],

        [[-0.3291, -0.7238, -0.3968, -0.0429,  0.1484,  0.8858],
         [-0.9697,  0.9750,  1.4834, -0.2072,  1.8970, -1.1792]]])


### All arguments in PyTorch are supported

In [3]:
a = pp.randn_SO3(3, device="cuda:0", dtype=torch.double, requires_grad=True)
b = pp.identity_like(a, device="cpu")
a, b
t = a.float()
a, t

(SO3Type Group:
 tensor([[ 0.0534,  0.7060, -0.2022,  0.6767],
         [-0.3990, -0.8116, -0.0492,  0.4239],
         [ 0.0213,  0.3176, -0.2628,  0.9108]], device='cuda:0',
        dtype=torch.float64, requires_grad=True),
 SO3Type Group:
 tensor([[ 0.0534,  0.7060, -0.2022,  0.6767],
         [-0.3990, -0.8116, -0.0492,  0.4239],
         [ 0.0213,  0.3176, -0.2628,  0.9108]], device='cuda:0',
        grad_fn=<AliasBackward0>))

# 2. Slicing and Shaping

In [4]:
A = pp.randn_SO3(2,2)
B = pp.randn_SO3(2,1)
C = torch.cat([A,B], dim=1)         # Tensor cat
C[0,1] = pp.randn_SO3(1)            # Slicing set
D = C[1,:].Log()                    # Slicing get
E, F = torch.split(C, [1,2], dim=1) # Tensor split
print('A:', A.gshape)
print('B:', B.gshape)
print('C:', C.gshape)
print('D:', D.gshape)
print('E:', E.gshape)
print('F:', F.gshape)

A: torch.Size([2, 2])
B: torch.Size([2, 1])
C: torch.Size([2, 3])
D: torch.Size([3])
E: torch.Size([2, 1])
F: torch.Size([2, 2])


# 3. Basic Operations

In [5]:
(x * y.Exp()).Inv().Log()

se3Type Group:
tensor([[[-0.7216, -0.3705, -0.7914, -0.5841,  0.8191,  0.2729],
         [ 0.7464,  0.3715, -0.9468,  1.5123, -0.0142,  0.0490]],

        [[ 0.3291,  0.7238,  0.3968,  0.0429, -0.1484, -0.8858],
         [ 0.9697, -0.9750, -1.4834,  0.2072, -1.8970,  1.1792]]])

# 4. Adjoint Transforms

In [6]:
X = pp.randn_Sim3(6, dtype=torch.double)
a = pp.randn_sim3(6, dtype=torch.double)
b = X.AdjT(a)
print((X * b.Exp() - a.Exp() * X).abs().mean() < 1e-7)

tensor(True)


In [7]:
X = pp.randn_SE3(8)
a = pp.randn_se3(8)
b = X.Adj(a)
print((b.Exp() * X - X * a.Exp()).abs().mean() < 1e-7)

tensor(True)


# 5. Grdients

In [8]:
x = pp.randn_so3(3, sigma=0.1, requires_grad=True, device="cuda")
assert x.is_leaf
loss = (x.Exp().Log()**2).sin().sum() # Just test, No physical meaning
loss.backward()
y = x.detach()
loss, x.grad, x, y

(tensor(0.1452, device='cuda:0', grad_fn=<SumBackward0>),
 tensor([[-0.1873, -0.3202, -0.0706],
         [-0.1536,  0.3928, -0.4324],
         [-0.1768, -0.0231, -0.2022]], device='cuda:0'),
 so3Type Group:
 tensor([[-0.0937, -0.1602, -0.0353],
         [-0.0768,  0.1965, -0.2164],
         [-0.0884, -0.0116, -0.1011]], device='cuda:0', requires_grad=True),
 so3Type Group:
 tensor([[-0.0937, -0.1602, -0.0353],
         [-0.0768,  0.1965, -0.2164],
         [-0.0884, -0.0116, -0.1011]], device='cuda:0'))

# 6. Test a Module

In [9]:
from torch import nn

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

class TestNet(nn.Module):
    def __init__(self, n):
        super().__init__()
        self.weight = pp.Parameter(pp.randn_so3(n))

    def forward(self, x):
        return self.weight.Exp() * x


n,epoch = 4, 5
net = TestNet(n).cuda()

optimizer = torch.optim.SGD(net.parameters(), lr = 0.2, momentum=0.9)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[2,4], gamma=0.5)

print("Before Optimization:\n", net.weight)
for i in range(epoch):
    optimizer.zero_grad()
    inputs = pp.randn_SO3(n).cuda()
    outputs = net(inputs)
    loss = outputs.abs().sum()
    loss.backward()
    optimizer.step()
    scheduler.step()
    print(loss)

print("Parameter:", count_parameters(net))
print("After Optimization:\n", net.weight)

Before Optimization:
 so3Type Group:
Parameter containing:
tensor([[-2.1001, -1.2749,  0.3482],
        [-1.4677, -0.8262,  0.6530],
        [-1.4498, -0.1315,  0.2014],
        [ 0.9867, -0.3606,  0.7121]], device='cuda:0', requires_grad=True)
tensor(6.6041, device='cuda:0', grad_fn=<SumBackward0>)
tensor(7.3226, device='cuda:0', grad_fn=<SumBackward0>)
tensor(6.6431, device='cuda:0', grad_fn=<SumBackward0>)
tensor(6.5915, device='cuda:0', grad_fn=<SumBackward0>)
tensor(7.0237, device='cuda:0', grad_fn=<SumBackward0>)
Parameter: 12
After Optimization:
 so3Type Group:
Parameter containing:
tensor([[-0.7197, -0.5055,  0.6865],
        [ 0.2213,  0.1932,  0.5925],
        [-0.9299, -0.3800, -0.2991],
        [ 0.1170, -0.1062,  0.5903]], device='cuda:0', requires_grad=True)
